## Static Heterogeneous Market Experiments
The heterogeneously populated market sessions are meant to simulate PRDE performance in a more realistic trading exchange. Therefore in this notebook I run multiple market sessions populated symetrically with multiple types of traders for realism including PRDE (30B/30S). The sessions implement a Poisson distribution order schedule ("timemode") like a real market, fixed spacing in the price distribution ("stepmode") for reduced randomness and increased reproducibility. Each market session lasts 5 min, but could be changed to last for example the same time as a trading day at the London Stock Exchange (8h 30min). 
The PRDE traders have varying k and F parameter values. Each combination is tested 10 times. The k ranges from 4 to 1024 in powers of 2 and the F lies between 0 and 2 in intervals of 0.2.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
import random

import scipy as sp
from scipy import stats
import seaborn as sns 
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import pandas as pd

import BSE
import helpers

In [8]:
def static_heterogeneous_session(k, F, trial_id_str, csv_str):
    sellers_spec = [('PRDE', 5, {'k': k, 's_min': -1.0, 's_max': +1.0, 'F': F}), ('ZIC', 5), ('ZIP', 5), ('GVWY', 5), ('SHVR', 5) , ('SNPR', 5)]
    buyers_spec = sellers_spec
    traders_spec = {'sellers':sellers_spec, 'buyers':buyers_spec}
    sup_range = (100, 200)
    dem_range = sup_range

    start_time = 0
    end_time = 60 * 5
    supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
    demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]

    order_interval = 5
    order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}

    trial_id = trial_id_str
    tdump = open(csv_str,'w')
    dump_all = False
    verbose = False

    BSE.market_session(trial_id, start_time, end_time, traders_spec, order_sched, tdump, dump_all, verbose)
    tdump.close()

In [3]:
k_range = np.logspace(1, 10, num=10, base=2)
k_range = k_range[1:10]
k_range = k_range.astype(int)
F_range = np.linspace(0.0, 2.0, 10)

In [9]:
def loop_kF(n, m, trial_id_str, csv_str):
    i = 4
    for i in n:
        for j in m:
            static_heterogeneous_session(i, j, trial_id_str + '_' + str(i) + '_' + str(j)[0:3], csv_str + '_' + str(i) + '_' + str(j)[0:3] + '.csv') 

In [10]:
random.seed(2)
i = 0
while i < 10:
    loop_kF(k_range, F_range, 'shet_' + str(i), 'shet_avg_bal_' + str(i))
    i = i+1